In [1]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras
from keras.preprocessing import image
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils

import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Using TensorFlow backend.


# Cropping the videos of Walking through the door from all the videos

In [2]:
import os, shutil
# The path to the directory with uncompressed original dataset 
base_dir = 'F:\DLProj\Charades_v4'

In [4]:
os.mkdir(base_dir)

In [8]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [10]:
# Directory with our training videos of door
train_door_dir = os.path.join(train_dir, 'door')
os.mkdir(train_door_dir)

# Directory with our training videos of not_door
train_non_door_dir = os.path.join(train_dir, 'not_door')
os.mkdir(train_non_door_dir)

# Directory with our validation videos of door
validation_door_dir = os.path.join(validation_dir, 'door')
os.mkdir(validation_door_dir)

# Directory with our validation videos of not_door
validation_non_door_dir = os.path.join(validation_dir, 'not_door')
os.mkdir(validation_non_door_dir)

# Directory with our validation videos of door
test_door_dir = os.path.join(test_dir, 'door')
os.mkdir(test_door_dir)

# Directory with our validation videos of not_door
test_non_door_dir = os.path.join(test_dir, 'not_door')
os.mkdir(test_non_door_dir)

In [3]:
#Reading the CSV file of the train set.
csv_dir = 'F:\DLProj\Charades\Charades_v1_train.csv'
df = pd.read_csv(csv_dir)

In [4]:
#Changing the index of the file to the File ID
#df.set_index('id')
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38


In [6]:
df_1 = df[(df.quality<6)]
df_1 = df_1[(df_1.verified == 'Yes')]
df_1

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
9,JSUF4,YMXV,Basement (A room below the ground floor),4.0,5.0,Yes,A person is sweeping up with a broom and putti...,broom;bucket;cup/glass/bottle;floor;sofa;water,A person is cleaning the floor with a brush in...,c123 16.10 32.00;c106 17.70 31.80;c102 0.00 12...,30.54
20,CUCBN,WQ8Z,Living room,4.0,6.0,Yes,A person smiles at a bag of groceries in the l...,bag;couch;doorway;sofa;table,A person is sitting on a couch and looks into ...,c154 7.40 12.90;c123 0.00 11.30;c009 3.70 10.9...,20.88
25,OBTP8,YMXV,Bedroom,4.0,7.0,Yes,A person opens a door and enters a bedroom. Th...,bed;box;clothes;door;mirror;shirt,"A person is walking, opens a door and looks in...",c008 3.50 11.80;c040 17.90 24.40;c096 9.20 17....,31.08
35,SDXZ8,HR43,Kitchen,5.0,7.0,Yes,A person walks into their kitchen and starts t...,dish;floor;plate;sink;vacuum,The person walks into the kitchen and starts t...,c137 35.60 52.00;c138 33.50 38.80;c118 11.60 2...,51.42
37,U79C9,9PLL,Stairs,5.0,7.0,Yes,A person sits on stairs and takes off their sh...,blanket;shoe;stairs,A person is sitting on some stairs taking off ...,c072 20.00 36.00;c073 15.30 27.00;c057 3.20 17.50,34.88
...,...,...,...,...,...,...,...,...,...,...,...
7958,OZSB3,KFGP,Home Office / Study (A room in a house used fo...,5.0,7.0,Yes,A person is sitting in their home office watch...,backpack;bag;chair;man;pillow;table;television;tv,"a man watching tv sitting in a chair, and grab...",c076 18.60 32.00;c079 18.60 27.80;c020 19.00 2...,30.92
7961,MOQIU,WQ8Z,Recreation room / Man cave,5.0,7.0,Yes,A person is laughing as the watch tv. The pers...,doorway;laptop;phone;sofa;table;television,A person sits on a couch watching and laughing...,c018 13.50 20.00;c019 16.80 30.00;c131 0.00 20...,28.62
7967,L1O0N,WG9D,Laundry room,3.0,3.0,Yes,A person in their laundry room begins undressi...,clothes;cup;pillow,"A person undresses, smashes their head into a ...",c076 2.80 9.20;c106 6.30 13.00;c107 6.00 13.00...,11.67
7974,LD011,BYF9,Entryway (A hall that is generally located at ...,5.0,7.0,Yes,A person in their entryway dressing to go outs...,book;clothes;coffee;cup;jacket;notepad,The person puts on a striped jacket and then w...,c110 30.20 37.00;c030 30.20 37.00;c148 0.00 30.30,35.50


In [82]:
#Dropping the irrelevant columns. (not needed in the current setup though)
df_2 = df_1.drop(['subject', 'quality','relevance','verified','script','objects','descriptions'], axis =1)
df_2.head()

,id,actions,length
9,JSUF4,c123 16.10 32.00;c106 17.70 31.80;c102 0.00 12...,30.54
20,CUCBN,c154 7.40 12.90;c123 0.00 11.30;c009 3.70 10.9...,20.88
25,OBTP8,c008 3.50 11.80;c040 17.90 24.40;c096 9.20 17....,31.08
41,78XZ9,c025 27.30 32.40;c026 0.00 33.00;c027 6.30 20....,31.79
48,PN4MI,c145 3.60 13.80;c065 4.90 13.10;c149 15.60 29....,30.50


In [11]:
len(df_1)

2784

In [13]:
# The file path can be changed to the directory where you want to save the file. 
df_1.to_csv('F:/DLProj/Charades_v4/filtered_train.csv', encoding='utf-8', index=False)

#  Extracting the file names of Door and Non-Door Videos on Train Set

In [14]:
# Function for parsing the Charades CSV
def parse_charades_csv(filename):
    labels = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            vid = row['id']
            actions = row['actions']
            if actions == '':
                actions = []
            else:
                actions = [a.split(' ') for a in actions.split(';')]
                actions = [{'class': x, 'start': float(
                    y), 'end': float(z)} for x, y, z in actions]
            labels[vid] = actions
    return labels

In [16]:
import csv
labels = parse_charades_csv('F:/DLProj/Charades_v4/filtered_train.csv')

In [17]:
door_f = {} #Saves all the file names, start and end times of the class for Walking through Door
non_door = {} #Saves all the file names, start and end times of all other classes
for a in labels.keys():  # a gets the file names
    for k in labels[a]:  # k gets all the embedded lists
        for key, value in k.items():   # key and value get's all the keys and values in the list 
            if key == 'class' and value =='c097':
                door_f.setdefault(a,[]).append(k)
            else:
                fn = a+str(value)
                non_door.setdefault(fn,[]).append(k)

# Code for extracting the Door videos of train set

In [19]:
import os
import ffmpeg
from moviepy.video.io.VideoFileClip import VideoFileClip

In [20]:
i = 0
d = []
for a in door_f.keys():#This gives the file names. 
    p1 = "F:\DLProj\Charades_v1\\"
    p2 = 'F:\\DLProj\\Charades_v5\\Cropped\\door'
    a1 = a + '.mp4'
    in_name = os.path.join(p1,  a1) #This gives the complete path of the file.
    out_name = os.path.join(p2, a1) #This gives the path for the target file
    for dicts in door_f[a]:
        for key, value in dicts.items():
            if key =='start':
                t1 = int(value)
            elif key =='end':
                t2 = int(value)
        if t2-t1<15: #Only taking 15 sec clips
            input_video_path = in_name
            output_video_path = out_name
            with VideoFileClip(input_video_path) as video:
                if t2 > (video.duration):
                    t2 = int(video.duration)
                new = video.subclip(t1, t2)
                new.write_videofile(output_video_path)
            # And saving them in Train/Door folder.
            d.append(a1) #This list would have all the file names in the train set

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [21]:
# Saving the file names of the training set of Class door
f = pd.Series(d)
f.to_csv('F:\\DLProj\\Charades_v5\\Cropped\\door\\list.csv', encoding='utf-8', index = False )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


# Code for cropping videos of Train Set (Not-Door)

In [38]:
i = 0
j = 0
d = []
for a in non_door.keys():#This gives the file names. 
    j +=1
    p1 = "F:\DLProj\Charades_v1\\"
    p2 = 'F:\\DLProj\\Charades_v5\\Cropped\\not_door\\'
    a1 = a[:5] + '.mp4'
    in_name = os.path.join(p1,  a1) #This gives the complete path of the file.
    out_name = os.path.join(p2, a1) #This gives the path for the target file
    for dicts in non_door[a]:
        for key, value in dicts.items():
            if key =='start':
                t1 = value
            elif key =='end':
                t2 = value
        if t2-t1<15: #Only taking at max 15 sec clips
            # Since there are in total 16000 clips fitting the criteria, we will take every fortieth video in the sequence and save it
            input_video_path = in_name
            output_video_path = out_name
            if j%40 == 0: 
                with VideoFileClip(input_video_path) as video:
                    if t2 > (video.duration):
                        t2 = int(video.duration)
                    new = video.subclip(t1, t2)
                    new.write_videofile(output_video_path)
                # Saving them in Train/Door folder.
                d.append(a1) #This list would have all the file names in the train set

In [43]:
f = pd.Series(d)
f.to_csv('F:\\DLProj\\Charades_v5\\Cropped\\not_door\\list.csv', encoding='utf-8', index = False )

59.44


# Code for cropping videos of Train Set (Not-Door)

In [ ]:
path_1 = 'F:/DLProj/Charades_v5/test/filtered_test.csv'
labels_1 = parse_charades_csv(path_1)

In [24]:
t_door_f = {} #Saves all the file names, start and end times of the class for Walking through Door
t_non_door = {} #Saves all the file names, start and end times of all other classes
for a in labels_1.keys():  # a gets the file names
    for k in labels_1[a]:  # k gets all the embedded lists
        for key, value in k.items():   # key and value get's all the keys and values in the list 
            if key == 'class' and value =='c097':
                t_door_f.setdefault(a,[]).append(k)
            else:
                fn = a+str(value)
                t_non_door.setdefault(fn,[]).append(k)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [ ]:
i = 0
d = []
for a in t_door_f.keys():#This gives the file names. 
    p1 = "F:\DLProj\Charades_v1\\"
    p2 = 'F:\\DLProj\\Charades_v5\\test\\Cropped\\door\\'
    a1 = a + '.mp4'
    in_name = os.path.join(p1,  a1) #This gives the complete path of the file.
    out_name = os.path.join(p2, a1) #This gives the path for the target file
    for dicts in t_door_f[a]:
        for key, value in dicts.items():
            if key =='start':
                t1 = value
            elif key =='end':
                t2 = value
        if t2-t1<15: #Only taking 15 sec clips
            input_video_path = in_name
            output_video_path = out_name
            with VideoFileClip(input_video_path) as video:
                if t2 > (video.duration):
                    t2 = int(video.duration)
                new = video.subclip(t1, t2)
                new.write_videofile(output_video_path)
            # And saving them in Train/Door folder.
            d.append(a1) #This list would have all the file names in the train set           

In [45]:
f = pd.Series(d)
f.to_csv('F:\\DLProj\\Charades_v5\\test\\Cropped\\door\\list.csv', encoding='utf-8', index = False )

# Code for cropping videos of Test Set (Not-Door)

In [1]:
i = 0
j = 0
d = []
for a in t_non_door.keys():#This gives the file names. 
    j +=1
    p1 = "F:\DLProj\Charades_v1\\"
    p2 = 'F:\\DLProj\\Charades_v5\\test\\Cropped\\not_door\\'
    a1 = a[:5] + '.mp4'
    in_name = os.path.join(p1,  a1) #This gives the complete path of the file.
    out_name = os.path.join(p2, a1) #This gives the path for the target file
    for dicts in t_non_door[a]:
        for key, value in dicts.items():
            if key =='start':
                t1 = value
            elif key =='end':
                t2 = value
        if t2-t1<15: #Only taking 15 sec clips
            # Since there are in total 16000 clips fitting the criteria, we will take every fortieth video in the sequence and save it
            input_video_path = in_name
            output_video_path = out_name
            if j%50 == 0: 
                with VideoFileClip(input_video_path) as video:
                    if t2 > (video.duration):
                        t2 = int(video.duration)
                    new = video.subclip(t1, t2)
                    new.write_videofile(output_video_path)
                # Saving them in Train/Door folder.
                d.append(a1) #This list would have all the file names in the train set

NameError: name 't_non_door' is not defined

In [50]:
f = pd.Series(d)
f.to_csv('F:\\DLProj\\Charades_v5\\test\\Cropped\\not_door\\list.csv', encoding='utf-8', index = False )

In [30]:
t_door_f = {} #Saves all the file names, start and end times of the class for Walking through Door
t_non_door = {} #Saves all the file names, start and end times of all other classes
for a in labels_1.keys():  # a gets the file names
    for k in labels_1[a]:  # k gets all the embedded lists
        for key, value in k.items():   # key and value get's all the keys and values in the list 
            if key == 'class' and value =='c097':
                t_door_f.setdefault(a,[]).append(k)
            else:
                fn = a+str(value)
                t_non_door.setdefault(fn,[]).append(k)